In [ ]:
%run ./../data/load-dataset.ipynb
%run ./../various/basic-neural-network.ipynb
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
ITERS = 100

RANDOM_SEED = 0

In [ ]:
def sparse_to_dense(sparse_bow_doc, dict_size):
    dense_bow_doc = np.zeros(dict_size).astype(int)
    dense_bow_doc[sparse_bow_doc.T[0]] = sparse_bow_doc.T[1]
    return dense_bow_doc


corpus, labels = df[[proc_doc_col, label_col]].T.values
X = np.array([sparse_to_dense(np.array(dictionary.doc2bow(doc)), len(dictionary)) for doc in corpus])
y = np.array([np.eye(2)[int(label)] for label in labels])
train_samples, test_samples, train_labels, test_labels = train_test_split(X, y, random_state=RANDOM_SEED)

In [ ]:
hidden_layer_size = 100
iterations = ITERS + 1
alpha = 1e-3
batch_size = 32  # set to 1 for stochastic gradient descent
hid_activation = Activation.RELU
out_activation = Activation.SIGMOID
dropout = 0.5
normalize = True
skip_remaining = True
is_generator = True
random_seed = RANDOM_SEED

model = BasicNeuralNetwork(hidden_layer_size, epochs=iterations, alpha=alpha,
                           batch_size=batch_size, skip_remaining=skip_remaining,
                           hid_activation=hid_activation, out_activation=out_activation, 
                           dropout=dropout, normalize=normalize,
                           is_generator=is_generator, random_seed=random_seed)

data = []
for _ in tqdm(model.fit(train_samples, train_labels), disable=SILENT, total=iterations):
    train_preds, test_preds = model.predict(train_samples), model.predict(test_samples)
    train_loss, train_accuracy = model.evaluate(train_preds, train_labels)
    test_loss, test_accuracy = model.evaluate(test_preds, test_labels)
    data.append({'Train Predictions': train_preds, 'Test Predictions': test_preds,
                 'Train Loss (MSE)': train_loss, 'Test Loss (MSE)': test_loss, 
                 'Train Accuracy': train_accuracy, 'Test Accuracy': test_accuracy})

results = pd.DataFrame(data)
results[['Train Loss (MSE)', 'Train Accuracy', 'Test Loss (MSE)', 'Test Accuracy']][::10].style \
            .highlight_min(subset=['Train Accuracy', 'Test Accuracy'], color='lightcoral') \
            .highlight_max(subset=['Train Accuracy', 'Test Accuracy'], color='lightgreen') \
            .highlight_min(subset=['Train Loss (MSE)', 'Test Loss (MSE)'], color='lightgreen') \
            .highlight_max(subset=['Train Loss (MSE)', 'Test Loss (MSE)'], color='lightcoral')

In [ ]:
normalize = 'true'

true_preds, true_labels = [np.argmax(pred) for pred in test_preds], \
                          [np.argmax(label) for label in test_labels]
ConfusionMatrixDisplay.from_predictions(true_labels, true_preds, normalize=normalize,
                                        cmap=plt.cm.Blues, display_labels=('reliable', 'unreliable'))